In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/job-dataset/unlabelled_development_set.csv
/kaggle/input/job-dataset/no_agument_training_5118.json
/kaggle/input/job-dataset/agument_work_5316.json
/kaggle/input/job-dataset/prompts_complete_test_salary.json
/kaggle/input/job-dataset/work_arrangements_development_set.csv
/kaggle/input/job-dataset/prompts_complete_test_work.json
/kaggle/input/job-dataset/work_arrangements_test_set.csv
/kaggle/input/job-dataset/combined_prompts_complete_6k.json
/kaggle/input/job-dataset/seniority_labelled_test_set.csv
/kaggle/input/job-dataset/agument_all_7998.json
/kaggle/input/job-dataset/agument_training_7800.json
/kaggle/input/job-dataset/seniority_labelled_development_set.csv
/kaggle/input/job-dataset/unlabelled_development_set.json
/kaggle/input/job-dataset/prompts_complete_test_seniority.json
/kaggle/input/job-dataset/salary_labelled_development_set.csv
/kaggle/input/job-dataset/test_1355.json
/kaggle/input/job-dataset/combined_prompts_complete_train_80.json
/kaggle/input/job-dataset

Note - Setting of this notebook model

LLM model: qwen0.5b

Training method: DAPT loading + prompt tuning

epoch:4

batch:8

learning rate: 3e-3

prompt_tuning_training set: /kaggle/input/job-dataset/agument_work_5316.json



In [ ]:
input_path = "/kaggle/input/job-dataset/agument_work_5316.json"
input_df = pd.read_json(input_path)
input_df

,prompt,complete
0,[TASK: Salary] Extract salary information from...,17500-17500-PHP-MONTHLY
1,[TASK: Salary] Extract salary information from...,16000-16000-PHP-MONTHLY
2,[TASK: Salary] Extract salary information from...,0-0-None-None
3,[TASK: Salary] Extract salary information from...,0-0-None-None
4,[TASK: Salary] Extract salary information from...,0-0-None-None
...,...,...
5311,[TASK: Work Arrangement] Extract work arrangem...,Remote
5312,[TASK: Work Arrangement] Extract work arrangem...,Hybrid
5313,[TASK: Work Arrangement] Extract work arrangem...,Hybrid
5314,[TASK: Work Arrangement] Extract work arrangem...,Hybrid


In [ ]:
import os
import time
import torch
from datasets import load_dataset
from transformers import (  # transformer
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model,
    PromptTuningConfig,
    TaskType
)
from peft import PeftModel

# 1. Environment Setting
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Setting GPU
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 2. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    trust_remote_code=True,
    use_fast=False,
    local_files_only=False
    )

2025-04-25 08:26:14.958158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745569575.186679      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745569575.254861      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
base_path = "/kaggle/input/dapt-model/transformers/default/1/dapt_model"
model = AutoModelForCausalLM.from_pretrained(base_path, trust_remote_code=True)

# # 5. Check trainable params (only prompt tokens should be trainable now)
# model.print_trainable_parameters()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:
# 3. 添加pad_token（如果不存在）
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 4. 配置Prompt Tuning
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Extract key information from the text:",  # 英文初始化文本
    num_virtual_tokens=20,
    tokenizer_name_or_path='Qwen/Qwen2.5-0.5B-Instruct',
    token_dim=model.config.hidden_size,
    num_transformer_submodules=1
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = get_peft_model(model, peft_config).to(device)

In [ ]:

def load_and_preprocess(data_path):
    dataset = load_dataset("json", data_files=data_path)

    dataset = dataset.filter(lambda x: len(x["prompt"]) > 0 and len(x["complete"]) > 0)

    def tokenize_function(examples):
        # print(examples)
        completions = [str(c) for c in examples["complete"]]
        texts = [p + c + tokenizer.eos_token for p, c in zip(examples["prompt"], completions)]

        tokenized = tokenizer(
            texts,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        prompt_tokenized = tokenizer(examples["prompt"], add_special_tokens=False)
        prompt_lengths = [len(ids) for ids in prompt_tokenized["input_ids"]]

        labels = tokenized["input_ids"].clone()
        for i, length in enumerate(prompt_lengths):
            if length >= 512:
                length = 511
            labels[i][:length] = -100

        tokenized["labels"] = labels
        return tokenized

    return dataset.map(tokenize_function, batched=True)

In [ ]:
def formal_train(json_path="salary_47_allin.json", target_model_path="./qwen_prompt_47"):
    dataset = load_and_preprocess(json_path)
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./qwen_prompt_tuning_output",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        num_train_epochs=4, # change this from 4 to 10
        learning_rate=3e-3, # change this from 3e-2 to 3e-3
        logging_dir="./logs",
        logging_steps=100,
        save_strategy="epoch",
        fp16=True,
        optim="adamw_torch",
        dataloader_num_workers=4,
        report_to="none"
    )

    # 7. 创建Trainer并训练
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        data_collator=data_collator,
    )

    print("开始训练...")
    start_time = time.time()
    trainer.train()
    print(f"训练完成，耗时: {time.time() - start_time:.2f}秒")

    model.save_pretrained(target_model_path)

    # To check memory useage
    # Optional: free unused memory before checking
    torch.cuda.empty_cache()
    # Memory usage check (in GB)
    print(f"Peak memory usage: {max_memory_allocated() / 1024**3:.2f} GB")



In [ ]:
formal_train(json_path=input_path, target_model_path="./qwen_05b_4epochs_dapt_prompt_fix_8batch_augwork")

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/5316 [00:00<?, ? examples/s]

Map:   0%|          | 0/5316 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


开始训练...


Step,Training Loss
100,2.949400
200,2.835100
300,2.816100
400,2.796300
500,2.781400
600,2.788200
700,2.786700
800,2.781800
900,2.771000
1000,2.778500


训练完成，耗时: 4297.81秒


NameError: name 'max_memory_allocated' is not defined

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

def generate_response(prompt):
  with torch.no_grad():
      inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
      outputs = model.generate(
          input_ids=inputs["input_ids"],
          attention_mask=inputs["attention_mask"],
          max_new_tokens=100,
          temperature=0.7,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )

      generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
      return tokenizer.decode(generated_tokens, skip_special_tokens=True)


In [ ]:
test_path = "/kaggle/input/job-dataset/test_1355.json"

# Load val_data from JSON file
import json

with open(test_path, 'r') as f: #Update with the path to your JSON File.
    data = json.load(f)

import time

t0 = time.time()
answers = []
for i, item in enumerate(data):
  # if (0 <= i < 20) or (567 <= i < 587) or (1256 <= i):
    # print("=" * 20, 'Round:', i, "=" * 20)
    p = item['prompt']
    # y = item['complete']
    # print("Target:", y)
    # print("\n Prompt tuning Answer:", generate_response(p))
    answers.append(generate_response(p))
t1 = time.time()
print("Inference time:", t1 - t0)

import pandas as pd

df = pd.DataFrame(data)
df["y_pred"] = answers

output_path = "/kaggle/working/df_output"
df.to_json(output_path, orient='records', indent=4, force_ascii=False)

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:1889: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


Inference time: 363.72235655784607
